In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
! conda install -c rdkit rdkit -y

--2021-12-04 07:06:14--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.1’

Miniconda3-py37_4.8 100%[===================>]  81.12M  87.4MB/s    in 0.9s    

2021-12-04 07:06:15 (87.4 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.1’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | / - \ | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::jpeg==9d=h7f8727e_0
  - defaults/linux-64::libuuid==1.0.3=h7f8727e_2
  - defaults/linux-64::numpy-base==1.21.2=py37h79a1101_0
  - defaults/linux-6

In [ ]:
# 导入包
import torch
import torch.optim as optim
from torch import nn as nn
import torch.nn.functional as F
from torch import sigmoid
from torch.autograd import Variable
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

In [ ]:
data = pd.read_csv('molnet_bace.csv')
data_sub = data[['mol', 'Class']]
train_data_x, test_data_x, train_y, test_y = train_test_split(data_sub['mol'], data_sub['Class'], test_size = 0.2, random_state = 1 )

# data process

In [ ]:
train_mols = [Chem.MolFromSmiles(smi) for smi in train_data_x]
train_morgan_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in train_mols]
train_morgan_fps_array = np.asarray(train_morgan_fps, dtype=float)
train_x_tensor = torch.from_numpy(train_morgan_fps_array)
train_y_tensor = torch.from_numpy(np.array(train_y))

In [ ]:
test_mols = [Chem.MolFromSmiles(smi) for smi in test_data_x]
test_morgan_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in test_mols]
test_morgan_fps_array = np.asarray(test_morgan_fps, dtype=float)
test_x_tensor = torch.from_numpy(test_morgan_fps_array)
test_y_tensor = torch.from_numpy(np.array(test_y))

# model

In [ ]:
class QSAR_mlp(nn.Module):  # 继承torch的module，即可以直接拥有 torch.nn.Module 的属性和方法
    def __init__(self):
        super(QSAR_mlp, self).__init__()
        self.fc1 = nn.Linear(2048, 524) # 对输入数据进行线性转换
        self.fc2 = nn.Linear(524, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10,2) 
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        output = sigmoid(self.fc4(h3))
        return output

In [ ]:
epochs = 25

model = QSAR_mlp()
losses = []
optimizer = optim.Adam(model.parameters(), lr=0.005)
for epoch in range(epochs):
    data, target = Variable(train_x_tensor).float(), Variable(train_y_tensor).long()
    optimizer.zero_grad()
    y_pred = model(data)
    loss = F.cross_entropy(y_pred, target)
    losses.append(loss.item())
    #if epoch % 10 == 0:
    print("Loss {}: {}".format(epoch, loss.item()))
    loss.backward()
    optimizer.step()

Loss 0: 0.6983951330184937
Loss 1: 0.6956345438957214
Loss 2: 0.6870374083518982
Loss 3: 0.6745038032531738
Loss 4: 0.6570096611976624
Loss 5: 0.6352607011795044
Loss 6: 0.6130855679512024
Loss 7: 0.592902660369873
Loss 8: 0.5768988132476807
Loss 9: 0.5644384622573853
Loss 10: 0.5545914173126221
Loss 11: 0.5462027788162231
Loss 12: 0.5395756363868713
Loss 13: 0.5328028202056885
Loss 14: 0.5262765288352966
Loss 15: 0.5198788642883301
Loss 16: 0.5135214924812317
Loss 17: 0.5077365040779114
Loss 18: 0.5009406805038452
Loss 19: 0.49396780133247375
Loss 20: 0.48721596598625183
Loss 21: 0.4795200824737549
Loss 22: 0.46983084082603455
Loss 23: 0.45994797348976135
Loss 24: 0.4509334862232208


# evaluate model

In [ ]:
pred_test_y_prob = model(Variable(test_x_tensor).float()).tolist()

torch_test_pred_list = []
for i, test_prob in enumerate(pred_test_y_prob):
    pred_pos_prob = test_prob[1]
    torch_test_pred_list.append(round(pred_pos_prob, 3))

torch_test_auc = round(roc_auc_score(test_y, torch_test_pred_list),2)
torch_test_acc = round(accuracy_score(test_y, np.round(torch_test_pred_list)),2)
torch_test_sen = round(recall_score(test_y, np.round(torch_test_pred_list)),2)
torch_test_spc = round((torch_test_acc * len(test_y)- torch_test_sen * test_y.sum())/(len(test_y)-test_y.sum()),2)


In [ ]:
pred_train_y_prob = model(Variable(train_x_tensor).float()).tolist()

torch_pred_list = []
for i, train_prob in enumerate(pred_train_y_prob):
    pred_pos_prob = train_prob[1]
    torch_pred_list.append(round(pred_pos_prob, 3))

torch_train_auc = round(roc_auc_score(train_y, torch_pred_list),2)
torch_train_acc = round(accuracy_score(train_y, np.round(torch_pred_list)),2)
torch_train_sen = round(recall_score(train_y, np.round(torch_pred_list)),2)
torch_train_spc = round((torch_train_acc * len(train_y)- torch_train_sen * train_y.sum())/(len(train_y)-train_y.sum()),2)



In [ ]:
torch_perf = {'AUC':[torch_train_auc, torch_test_auc],
            'ACC':[torch_train_acc, torch_test_acc],
            'SEN':[torch_train_sen, torch_test_sen],
            'SPC':[torch_train_spc, torch_test_spc]}

In [13]:
pd.DataFrame(torch_perf, index=['train', 'test'])

,AUC,ACC,SEN,SPC
train,0.95,0.93,0.96,0.90
test,0.89,0.81,0.90,0.73
